In [1]:
# 1

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import gc

In [81]:
def csv_to_parquet(csv_path, save_name):
    df = pd.read_csv(csv_path)
    df.to_parquet('./' + save_name + '.parquet')
    del df
    gc.collect()
    print(save_name, 'Done.')

In [82]:
# pyarrow 설치 해야함, 9.0.0
csv_to_parquet('C:/Users/knzss/Documents/ds_study/project/ds5-ml-repo-1/data/train.csv', 'train')
csv_to_parquet('C:/Users/knzss/Documents/ds_study/project/ds5-ml-repo-1/data/test.csv', 'test')

train Done.
test Done.


In [2]:
# 2

train = pd.read_parquet('./train.parquet')
test = pd.read_parquet('./test.parquet')

In [84]:
train

,id,base_date,day_of_week,base_hour,lane_count,road_rating,road_name,multi_linked,connect_code,maximum_speed_limit,...,road_type,start_node_name,start_latitude,start_longitude,start_turn_restricted,end_node_name,end_latitude,end_longitude,end_turn_restricted,target
0,TRAIN_0000000,20220623,목,17,1,106,지방도1112호선,0,0,60.0,...,3,제3교래교,33.427747,126.662612,없음,제3교래교,33.427749,126.662335,없음,52.0
1,TRAIN_0000001,20220728,목,21,2,103,일반국도11호선,0,0,60.0,...,0,광양사거리,33.500730,126.529107,있음,KAL사거리,33.504811,126.526240,없음,30.0
2,TRAIN_0000002,20211010,일,7,2,103,일반국도16호선,0,0,80.0,...,0,창고천교,33.279145,126.368598,없음,상창육교,33.280072,126.362147,없음,61.0
3,TRAIN_0000003,20220311,금,13,2,107,태평로,0,0,50.0,...,0,남양리조트,33.246081,126.567204,없음,서현주택,33.245565,126.566228,없음,20.0
4,TRAIN_0000004,20211005,화,8,2,103,일반국도12호선,0,0,80.0,...,0,애월샷시,33.462214,126.326551,없음,애월입구,33.462677,126.330152,없음,38.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4701212,TRAIN_4701212,20211104,목,16,1,107,-,0,0,50.0,...,0,대림사거리,33.422145,126.278125,없음,금덕해운,33.420955,126.273750,없음,20.0
4701213,TRAIN_4701213,20220331,목,2,2,107,-,0,0,80.0,...,3,광삼교,33.472505,126.424368,없음,광삼교,33.472525,126.424890,없음,65.0
4701214,TRAIN_4701214,20220613,월,22,2,103,일반국도12호선,0,0,60.0,...,0,고성교차로,33.447183,126.912579,없음,성산교차로,33.444121,126.912948,없음,30.0
4701215,TRAIN_4701215,20211020,수,2,2,103,일반국도95호선,0,0,80.0,...,0,제6광령교,33.443596,126.431817,없음,관광대학입구,33.444996,126.433332,없음,73.0


In [ ]:
# 전처리

In [3]:
# 3

# [요일, 시작 지점의 회전제한 유무, 도착지점의 회전제한 유무]
str_col = ['day_of_week','start_turn_restricted','end_turn_restricted']
for i in str_col:
    le = LabelEncoder()
    le=le.fit(train[i])
    train[i]=le.transform(train[i])
    
    for label in np.unique(test[i]):
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
            
    print(i, le.classes_)
    test[i]=le.transform(test[i])

day_of_week ['금' '목' '수' '월' '일' '토' '화']
start_turn_restricted ['없음' '있음']
end_turn_restricted ['없음' '있음']


In [5]:
# 4

y_train = train['target'] 

# [아이디, 날짜, 평균속도, 도로명, 시작지점명, 도착지점명, 통과제한차량]
X_train = train.drop(['id','base_date', 'target','road_name', 'start_node_name', 'end_node_name','vehicle_restricted'], axis=1)

test = test.drop(['id','base_date', 'road_name', 'start_node_name', 'end_node_name','vehicle_restricted'], axis=1)

print(X_train.shape)
print(y_train.shape)
print(test.shape)

(4701217, 16)
(4701217,)
(291241, 16)


주말, 공휴일?, 연후 시작일, 중간일, 종료일
https://nomalcy.tistory.com/303

In [21]:
X_train = train.drop(['id','base_date', 'target','road_name', 'start_node_name', 'end_node_name','vehicle_restricted'], axis=1)

In [30]:
# 요일, 시간대, 차로수, 도로등급, 중용구간 여부, 연결로 코드, 최고속도제한, 통과제한하중, 통과제한높이
# 도로유형, 시작지점위도, 시작지점경도, 시작지점의 회전제한 유무
# 도착지점위도, 도착지점경도, 도착지점의 회전제한 유무
# 16개

X_train.head(7)

,day_of_week,base_hour,lane_count,road_rating,multi_linked,connect_code,maximum_speed_limit,weight_restricted,height_restricted,road_type,start_latitude,start_longitude,start_turn_restricted,end_latitude,end_longitude,end_turn_restricted
0,1,17,1,106,0,0,60.0,32400.0,0.0,3,33.427747,126.662612,0,33.427749,126.662335,0
1,1,21,2,103,0,0,60.0,0.0,0.0,0,33.500730,126.529107,1,33.504811,126.526240,0
2,4,7,2,103,0,0,80.0,0.0,0.0,0,33.279145,126.368598,0,33.280072,126.362147,0
3,0,13,2,107,0,0,50.0,0.0,0.0,0,33.246081,126.567204,0,33.245565,126.566228,0
4,6,8,2,103,0,0,80.0,0.0,0.0,0,33.462214,126.326551,0,33.462677,126.330152,0
5,3,7,2,107,0,0,60.0,0.0,0.0,0,33.249949,126.505664,0,33.252183,126.506069,0
6,1,0,2,107,0,0,60.0,0.0,0.0,0,33.418412,126.268029,0,33.414175,126.269378,0


In [26]:
# 7월 데이터를 빼서 테스트데이터로 사용하면 좋을듯
train.sort_values('base_date')

,id,base_date,day_of_week,base_hour,lane_count,road_rating,road_name,multi_linked,connect_code,maximum_speed_limit,...,road_type,start_node_name,start_latitude,start_longitude,start_turn_restricted,end_node_name,end_latitude,end_longitude,end_turn_restricted,target
430379,TRAIN_0430379,20210901,2,16,3,106,지방도1132호선,0,0,70.0,...,3,진동1로,33.518987,126.568861,1,동중입구,33.519726,126.571488,0,37.0
369019,TRAIN_0369019,20210901,2,11,1,103,일반국도16호선,0,0,50.0,...,0,모살물교,33.312509,126.703863,0,한남교,33.312449,126.696173,0,36.0
1132647,TRAIN_1132647,20210901,2,4,2,103,일반국도12호선,0,0,80.0,...,0,애월샷시,33.462214,126.326551,0,애월입구,33.462677,126.330152,0,43.0
4543127,TRAIN_4543127,20210901,2,16,1,107,-,0,0,60.0,...,0,서중2교,33.331742,126.676431,0,한남교차로,33.325096,126.665698,0,55.0
3428956,TRAIN_3428956,20210901,2,9,1,103,일반국도16호선,0,0,50.0,...,3,월산2교,33.260056,126.490987,0,월산2교,33.259995,126.491306,0,53.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2053638,TRAIN_2053638,20220731,4,21,1,107,중문로,0,0,50.0,...,0,신양약방,33.251716,126.425612,0,중문한의원,33.251884,126.423200,0,21.0
2748124,TRAIN_2748124,20220731,4,12,3,107,연삼로,0,0,70.0,...,0,제2거로교,33.508464,126.558231,0,거로사거리,33.507349,126.556173,1,36.0
2447938,TRAIN_2447938,20220731,4,12,2,107,경찰로,0,0,60.0,...,0,신성교회,33.253074,126.506393,0,서호2차현대맨션203동,33.252183,126.506069,0,24.0
3749761,TRAIN_3749761,20220731,4,1,2,103,일반국도16호선,0,0,60.0,...,0,서홍교,33.264112,126.554043,0,서흥동사무소,33.263685,126.550978,0,37.0


In [27]:
test.head(10)

,id,base_date,day_of_week,base_hour,lane_count,road_rating,road_name,multi_linked,connect_code,maximum_speed_limit,...,height_restricted,road_type,start_node_name,start_latitude,start_longitude,start_turn_restricted,end_node_name,end_latitude,end_longitude,end_turn_restricted
0,TEST_000000,20220825,1,17,3,107,연삼로,0,0,70.0,...,0.0,0,산지2교,33.499427,126.541298,0,제주은행사거리,33.500772,126.543837,1
1,TEST_000001,20220809,6,12,2,103,일반국도12호선,0,0,70.0,...,0.0,3,중문입구,33.258507,126.427003,0,관광단지입구,33.258119,126.415840,0
2,TEST_000002,20220805,0,2,1,103,일반국도16호선,0,0,60.0,...,0.0,0,도순3교,33.258960,126.476508,0,도순2교,33.259206,126.474687,0
3,TEST_000003,20220818,1,23,3,103,일반국도11호선,0,0,70.0,...,0.0,0,아라주공아파트,33.473494,126.545647,0,인다마을,33.471061,126.545467,0
4,TEST_000004,20220810,2,17,3,106,번영로,0,0,70.0,...,0.0,0,부록교 시종점,33.501477,126.569223,0,봉개교 시종점,33.496863,126.581230,0
5,TEST_000005,20220803,2,1,2,103,일반국도12호선,0,0,60.0,...,0.0,0,외도1동,33.493036,126.430946,0,외도연대마을,33.492699,126.428241,0
6,TEST_000006,20220822,3,4,1,107,수영장길,0,0,60.0,...,0.0,0,제2오라교,33.498363,126.516407,0,한국병원앞,33.499973,126.516469,1
7,TEST_000007,20220809,6,5,2,107,-,0,0,80.0,...,0.0,0,해안교,33.466433,126.454583,0,해안교,33.466607,126.454022,0
8,TEST_000008,20220818,1,5,2,106,지방도1136호선,0,0,60.0,...,0.0,0,서귀포시산림조합,33.260382,126.513153,0,청룡개발,33.260513,126.514851,0
9,TEST_000009,20220828,4,3,3,107,제2거로교,0,0,70.0,...,0.0,3,제2거로교,33.508712,126.558703,0,제2거로교,33.508464,126.558231,0


In [36]:
# 주말(토, 일) - 요일컬럼으로 알수있음, 연휴(3일이상, 시작, 중간, 끝), 단독 굥휴일를 체크하는 컬럼을 만듦?
# pip install pytimeker 0.1.0

from pytimekr import pytimekr

kr = pytimekr.

In [69]:
# 1월 1일, 설날, 삼일절, 어린이날, 부처님오신날, 현충일, 광복절, 추석, 개천절, 한글날, 크리스마스
# 대체 : 설, 추석, 어린이날, 한글날, 광복절, 삼일절
int(''.join(str(kr[0]).split('-')))

20220131

In [41]:
from datetime import datetime, date
# 0이 월
today = datetime(2022, 10, 26)   
print(today.weekday())


2


In [72]:
int(train['base_date'][0])

20220623

In [70]:
## 월별 데이터 month사용

da = str(train['base_date'][0])

year = int(da[: 4])
month = int(da[4 : 6])
day = int(da[6 :])

today = datetime(year, month, day)   
print(today.weekday())

3


In [90]:
# 20221022 정수형태로 리스트에 정리

df_year = 2022

chuseok = []
lunar_newyear = []
y_list = []
x_list = []

# 대체 있음
c = pytimekr.red_days(pytimekr.chuseok(year = df_year))                     # 추석
l = pytimekr.red_days(pytimekr.lunar_newyear(year = df_year))               # 설날

for idx, item in enumerate(c):
    chuseok.append(int(''.join(str(item).split('-'))))
    
for idx, item in enumerate(l):
    lunar_newyear.append(int(''.join(str(item).split('-'))))


y_list.append(int(''.join(str(pytimekr.hangul(year = df_year)).split('-'))))                      # 한글날
y_list.append(int(''.join(str(pytimekr.children(year = df_year)).split('-'))))                    # 어린이날
y_list.append(int(''.join(str(pytimekr.independence(year = df_year)).split('-'))))                # 광복절
y_list.append(int(''.join(str(pytimekr.samiljeol(year = df_year)).split('-'))))                   # 삼일절


# 대체 없음
x_list.append(int(''.join(str(pytimekr.memorial(year = df_year)).split('-'))))                    # 현충일
x_list.append(int(''.join(str(pytimekr.buddha(year = df_year)).split('-'))))                      # 석가탄신일
x_list.append(int(''.join(str(pytimekr.constitution(year = df_year)).split('-'))))                # 제헌절
x_list.append(int(''.join(str(pytimekr.christmas(year = df_year)).split('-'))))                   # 크리스마스
x_list.append(int(''.join(str(pytimekr.newyear(year = df_year)).split('-'))))                     # 1월 1일

In [91]:
holidays_dic = {
                    'chuseok' : chuseok,
                    'lunar_newyear' : lunar_newyear,
                    'y_list' : y_list,
                    'x_list' : x_list
}

In [92]:
holidays_dic

{'chuseok': [20220909, 20220910, 20220911],
 'lunar_newyear': [20220131, 20220201, 20220202],
 'y_list': [20221009, 20220505, 20220815, 20220301],
 'x_list': [20220606, 20220508, 20220717, 20221225, 20220101]}

In [ ]:
# 날짜를 뽑아서 날짜를 공휴일과 비교?
# 뽑은 공휴일을 날짜에 비교?

day_data = 20220131

for key in list(holidays_dic.keys()):
        if day_data in holidays_dic[key]:
            if key == 'x_list':
                print('대체 X 단독 공휴일')
            elif key == 'y_list':
                if 요일데이터 == 주말:
                    월요일을 대체공휴일로
                print('대체 Y 단독 공휴일')
            else:
                if 요일데이터 == holidays_dic[key][1]:
                    토요일이면 월요일을 대체공휴일로
                    월요일이면 화요일을 대체공휴일로
                
                holidays_dic[key][0]과 holidays_dic[key][2]는 연휴 시작과 끝?
            
        요일데이터

In [4]:
# 2만건으로 학습
# 1천건으로 테스트
from sklearn.utils import shuffle

# tr = shuffle(train.sort_values('base_date')[train['base_date'] < 20220701]).iloc[:20000]
# te = shuffle(train.sort_values('base_date')[train['base_date'] > 20220631]).iloc[:1000]


# 얼마나 균일하게 뽑히는지
tr = shuffle(train.sort_values('base_date')[train['base_date'] < 20220701])
te = shuffle(train.sort_values('base_date')[train['base_date'] > 20220631])


# 가이드를 기준으로 유의해서 상관계수 비교
# 가이드에 drop에서 빠진 컬럼도 상관계수 비교
# y_train = tr['target']
# X_train = tr.drop(['id','base_date', 'target','road_name', 'start_node_name', 'end_node_name','vehicle_restricted'], axis=1)

# y_test = te['target']
# X_test = te.drop(['id','base_date', 'target','road_name', 'start_node_name', 'end_node_name','vehicle_restricted'], axis=1)

y_train = tr['target']
X_train = tr.drop(['day_of_week', 'multi_linked', 'connect_code', 'height_restricted', 'id','base_date', 'target','road_name', 'start_node_name', 'end_node_name','vehicle_restricted'], axis=1)

y_test = te['target']
X_test = te.drop(['day_of_week', 'multi_linked', 'connect_code', 'height_restricted', 'id','base_date', 'target','road_name', 'start_node_name', 'end_node_name','vehicle_restricted'], axis=1)


print(X_train.shape)
print(y_train.shape)
print(X_test.shape)

C:\Users\knzss\AppData\Local\Temp\ipykernel_1324\3133249128.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tr = shuffle(train.sort_values('base_date')[train['base_date'] < 20220701])
C:\Users\knzss\AppData\Local\Temp\ipykernel_1324\3133249128.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  te = shuffle(train.sort_values('base_date')[train['base_date'] > 20220631])


(4426522, 12)
(4426522,)
(274695, 12)


KeyError: False

In [10]:
from sklearn.metrics import mean_absolute_error


mae_list = []
min_num = 10
min_stack = 0
stop_num = 3

# test, max_depth
for i in range(-1, 20):
    LR = lgb.LGBMRegressor(random_state = 42,
                           max_depth = i
                          ).fit(X_train, y_train)

    pred = LR.predict(X_test)
    mae = mean_absolute_error(pred, y_test)
    
    if min_num > mae:
        min_stack = 0
        mae_list.append((i, mae))
        min_num = mae
        print('max_depth : ', i, ', mae : ', mae)
    elif min_stack != stop_num:
        min_stack += 1
        mae_list.append((i, mae))
        min_num = mae
        print('max_depth : ', i, ', mae : ', mae)
    elif min_stack == stop_num:
        break
            

max_depth :  -1 , mae :  5.072009186932158
max_depth :  0 , mae :  5.072009186932158
max_depth :  1 , mae :  8.580603462129307
max_depth :  2 , mae :  7.33697810484018
max_depth :  3 , mae :  6.634549924483167
max_depth :  4 , mae :  5.95700943277578
max_depth :  5 , mae :  5.54628457732585
max_depth :  6 , mae :  5.296210204212831
max_depth :  7 , mae :  5.214639608825163
max_depth :  8 , mae :  5.181666364796581
max_depth :  9 , mae :  5.1488795937694904
max_depth :  10 , mae :  5.064579257092972
max_depth :  11 , mae :  5.065714234188544
max_depth :  12 , mae :  5.0885120176507765
max_depth :  13 , mae :  5.056857609618951
max_depth :  14 , mae :  5.057668678260776
max_depth :  15 , mae :  5.027391340213928
max_depth :  16 , mae :  5.06558649606788
max_depth :  17 , mae :  5.047917925365594
max_depth :  18 , mae :  5.078356972172137
max_depth :  19 , mae :  5.040795633603219


In [13]:
# test, num_leaves 디폴트 31, 10부터 50까지 진행
default = 100
max_num_t = 150

for i in range(default, max_num_t + 1):
    LR = lgb.LGBMRegressor(random_state = 42,
                           num_leaves  = i
                          ).fit(X_train, y_train)

    pred = LR.predict(X_test)
    mae = mean_absolute_error(pred, y_test)
    
    if min_num > mae:
        min_stack = 0
        mae_list.append((i, mae))
        min_num = mae
        print('num_leaves  : ', i, ', mae : ', mae)
    elif min_stack != stop_num:
        min_stack += 1
        mae_list.append((i, mae))
        min_num = mae
        print('num_leaves  : ', i, ', mae : ', mae)
    elif min_stack == stop_num:
        break

num_leaves  :  100 , mae :  4.337545905950325
num_leaves  :  101 , mae :  4.323674945705277
num_leaves  :  102 , mae :  4.321046023494213
num_leaves  :  103 , mae :  4.301130512028979
num_leaves  :  104 , mae :  4.321720692993959
num_leaves  :  105 , mae :  4.298069014346037
num_leaves  :  106 , mae :  4.3036836625923875
num_leaves  :  107 , mae :  4.311829598870999
num_leaves  :  108 , mae :  4.288106714009215
num_leaves  :  109 , mae :  4.302169306520981
num_leaves  :  110 , mae :  4.276508022741148
num_leaves  :  111 , mae :  4.27999127383573
num_leaves  :  112 , mae :  4.281033388583948
num_leaves  :  113 , mae :  4.277851134524665
num_leaves  :  114 , mae :  4.266843367094763
num_leaves  :  115 , mae :  4.277371782159103
num_leaves  :  116 , mae :  4.273483525734047
num_leaves  :  117 , mae :  4.27013563519106
num_leaves  :  118 , mae :  4.262282141068192
num_leaves  :  119 , mae :  4.25449740557912
num_leaves  :  120 , mae :  4.2556898389151865
num_leaves  :  121 , mae :  4.26748

In [15]:
LR.feature_importances_

12

In [7]:
i = 0
print('max_depth : ', i, ', test : ', i)

max_depth :  0 , test :  0


In [8]:
len(X_train.columns)

16

In [47]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4701217 entries, 0 to 4701216
Data columns (total 23 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   id                     object 
 1   base_date              int64  
 2   day_of_week            int32  
 3   base_hour              int64  
 4   lane_count             int64  
 5   road_rating            int64  
 6   road_name              object 
 7   multi_linked           int64  
 8   connect_code           int64  
 9   maximum_speed_limit    float64
 10  vehicle_restricted     float64
 11  weight_restricted      float64
 12  height_restricted      float64
 13  road_type              int64  
 14  start_node_name        object 
 15  start_latitude         float64
 16  start_longitude        float64
 17  start_turn_restricted  int32  
 18  end_node_name          object 
 19  end_latitude           float64
 20  end_longitude          float64
 21  end_turn_restricted    int32  
 22  target            

In [66]:
ttt = tr.drop(['id','road_name', 'start_node_name', 'end_node_name','vehicle_restricted'], axis=1)
ttt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4426522 entries, 577878 to 3305650
Data columns (total 18 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   base_date              int64  
 1   day_of_week            int32  
 2   base_hour              int64  
 3   lane_count             int64  
 4   road_rating            int64  
 5   multi_linked           int64  
 6   connect_code           int64  
 7   maximum_speed_limit    float64
 8   weight_restricted      float64
 9   height_restricted      float64
 10  road_type              int64  
 11  start_latitude         float64
 12  start_longitude        float64
 13  start_turn_restricted  int32  
 14  end_latitude           float64
 15  end_longitude          float64
 16  end_turn_restricted    int32  
 17  target                 float64
dtypes: float64(8), int32(3), int64(7)
memory usage: 591.0 MB


In [67]:
tar_corr = ttt.corr()[abs(ttt.corr()) > 0.05]

In [68]:
tar_corr.iloc[-1]

base_date                     NaN
day_of_week                   NaN
base_hour               -0.158596
lane_count              -0.141434
road_rating             -0.260453
multi_linked                  NaN
connect_code                  NaN
maximum_speed_limit      0.432182
weight_restricted        0.290456
height_restricted             NaN
road_type                0.198994
start_latitude                NaN
start_longitude               NaN
start_turn_restricted   -0.156077
end_latitude                  NaN
end_longitude                 NaN
end_turn_restricted     -0.162068
target                   1.000000
Name: target, dtype: float64

In [64]:
# 0.05보다 낮은 데이터 컬럼 drop

p = tr.drop(['day_of_week', 'multi_linked', 'connect_code', 'height_restricted', 'id','base_date', 'road_name', 'start_node_name', 'end_node_name','vehicle_restricted'], axis=1)

In [65]:
p.corr()

,base_hour,lane_count,road_rating,maximum_speed_limit,weight_restricted,road_type,start_latitude,start_longitude,start_turn_restricted,end_latitude,end_longitude,end_turn_restricted,target
base_hour,1.000000,-0.029529,0.032695,-0.038158,-0.003741,-0.007942,-0.022165,-0.011810,-0.012478,-0.022160,-0.011821,-0.012614,-0.158596
lane_count,-0.029529,1.000000,-0.100050,0.382190,-0.176104,-0.055785,0.172184,-0.103655,0.294704,0.171851,-0.103584,0.314053,-0.141434
road_rating,0.032695,-0.100050,1.000000,-0.328717,-0.116102,-0.121890,-0.197821,0.005962,-0.025326,-0.197877,0.005950,-0.025738,-0.260453
maximum_speed_limit,-0.038158,0.382190,-0.328717,1.000000,0.087402,0.060384,0.245347,-0.036321,0.090746,0.245173,-0.036231,0.100848,0.432182
weight_restricted,-0.003741,-0.176104,-0.116102,0.087402,1.000000,0.803042,-0.128324,0.036524,-0.119463,-0.128337,0.036512,-0.119327,0.290456
road_type,-0.007942,-0.055785,-0.121890,0.060384,0.803042,1.000000,-0.049516,0.035557,-0.103457,-0.049525,0.035535,-0.103281,0.198994
start_latitude,-0.022165,0.172184,-0.197821,0.245347,-0.128324,-0.049516,1.000000,0.127601,0.275823,0.999041,0.127561,0.276142,0.036461
start_longitude,-0.011810,-0.103655,0.005962,-0.036321,0.036524,0.035557,0.127601,1.000000,0.041304,0.127447,0.999241,0.041119,0.000786
start_turn_restricted,-0.012478,0.294704,-0.025326,0.090746,-0.119463,-0.103457,0.275823,0.041304,1.000000,0.276193,0.041045,0.411219,-0.156077
end_latitude,-0.022160,0.171851,-0.197877,0.245173,-0.128337,-0.049525,0.999041,0.127447,0.276193,1.000000,0.127653,0.276078,0.036319


In [ ]:
https://nurilee.com/2020/04/03/lightgbm-definition-parameter-tuning/

In [ ]:
from sklearn.metrics import mean_absolute_error

LR = lgb.LGBMRegressor(random_state=42).fit(X_train, y_train)

pred = LR.predict(X_test)